In [54]:
%load_ext sql
%sql sqlite:///customer_personality.db


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [55]:
import pandas as pd
import sqlite3

df = pd.read_csv('marketing_campaign.csv', sep='\t')

# Step 2: Connect to your SQLite database
conn = sqlite3.connect("customer_personality.db")


# Step 3: Write the DataFrame into a SQL table
df.to_sql("customers", conn, if_exists="replace", index=False)


2240

In [56]:
import numpy as np

# Add synthetic columns to match expected ML structure
df_customers = df.copy()
df_customers['customer_id'] = df_customers['ID']
df_customers['contract_type'] = np.random.choice(['Month-to-Month', 'One Year', 'Two Year'], size=len(df))
df_customers['monthly_spend'] = np.random.normal(loc=60, scale=15, size=len(df)).round(2)
df_customers['last_active'] = pd.to_datetime('2024-01-01') - pd.to_timedelta(np.random.randint(30, 1000, size=len(df)), unit='D')

# Write to SQL
df_customers[['customer_id', 'contract_type', 'monthly_spend', 'last_active']].to_sql(
    'customers', conn, if_exists='replace', index=False
)


2240

In [57]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report



In [63]:
print("Test set size:", len(y_test))
print("Churn class distribution in test set:")
print(y_test.value_counts())
print("Total samples:", len(X))


Test set size: 1
Churn class distribution in test set:
churned
1    1
Name: count, dtype: int64
Total samples: 3


In [58]:
%%sql result_churn <<
SELECT
  c.customer_id,
  c.age,
  c.contract_type,
  c.monthly_spend,
  julianday('2025-01-01') - julianday(c.last_active) AS days_since_active,
  CASE
      WHEN julianday('2025-01-01') - julianday(c.last_active) > 180 THEN 1
      ELSE 0
  END AS churned,
  d.Income,
  d.Kidhome,
  d.Teenhome,
  d.Education,
  d.Marital_Status,
  d.Recency
FROM customers c
LEFT JOIN campaign_data d ON c.customer_id = d.ID;
# Convert SQL result to DataFrame
df_churn = result_churn.DataFrame()

 * sqlite:///customer_personality.db
(sqlite3.OperationalError) no such column: c.age
[SQL: SELECT
  c.customer_id,
  c.age,
  c.contract_type,
  c.monthly_spend,
  julianday('2025-01-01') - julianday(c.last_active) AS days_since_active,
  CASE
      WHEN julianday('2025-01-01') - julianday(c.last_active) > 180 THEN 1
      ELSE 0
  END AS churned,
  d.Income,
  d.Kidhome,
  d.Teenhome,
  d.Education,
  d.Marital_Status,
  d.Recency
FROM customers c
LEFT JOIN campaign_data d ON c.customer_id = d.ID;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [59]:
# Drop non-feature columns
X = df_churn.drop(columns=['customer_id', 'churned'])
y = df_churn['churned']

# One-hot encode
X = pd.get_dummies(X)

# Fill missing
X = X.fillna(0)

# Split, train, evaluate
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           1       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1

